<center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color:yellow ; border-radius:5px; font-size:250%; text-align:center;padding:3.0px; background: #3da859; border-bottom: 3px solid #2c3e50   ; border-top: 8px solid k; border-top: 8px solid k" > Electric Vehicles Registered in Washington State </center>
  
<center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: #3da859  ; border-radius:5px; font-size:180%; text-align:center;padding:3.0px; background: yellow; border-bottom: 6px solid #2c3e50   ; border-top: 8px solid k" > Where are they now </center>

<center>
<img src="https://img.freepik.com/premium-vector/electro-power-green-energy-symbol-linear-electro-cars-charge-charging-station_348818-1002.jpg?size=338&ext=jpg&ga=GA1.1.2008272138.1727222400&semt=ais_hybrid" width="1020"/>
</center>

# <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 1. Introduction </p>

This notebook is about electric vehicles (EV) registered in the US state of Washington. Our aim is to view where these vehicles are now. We expect the majority of the EV to still be in the state of Washington.
Tabulation and mapping will be used to bring out and show the findings.

# <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 2. Load Tools and Dataset </p>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import wkt
import seaborn as sns
import plotly.express as px
from pandas.plotting import scatter_matrix

import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import warnings
warnings.simplefilter('ignore')
elec_type_colors = ['green', 'gold']

In [ ]:
df_full = pd.read_csv('/kaggle/input/electric-vhicule-population-data/Electric_Vehicle_Population_Data.csv', index_col='DOL Vehicle ID')
df_full.head(2)

In [ ]:
print(f'The dataset shows that there have been {df_full.shape[0]} EV registered in the state of Washigton in the period covered.')

In [ ]:
print(f"The car model years span a period from {df_full['Model Year'].min()} to {df_full['Model Year'].max()}")

## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 2.2 Cleaning </p>


We will use short descriptive for both Electric Type and Clean Alternative Fuel Vehicle (CAFV) Eligibility.

    'Eligibility unknown as battery range has not been researched': 'not_verified_yet',
    'Clean Alternative Fuel Vehicle Eligible' : 'eligible',
    'Not eligible due to low battery range' : 'not eligible


In [ ]:
df_full.isna().sum().to_frame('number of NaN').style.bar(cmap='Reds')

In [ ]:
df_full['Legislative District'] = df_full['Legislative District'].map(lambda x: str(x).split('.')[0])

In [ ]:
df_full['Vehicle Location'] = df_full['Vehicle Location'].fillna(method='ffill')

The majority of the missing information, 442, is on Legislative District. This column will not be needed in this project. However, the missing values can be filled as shown below.

In [ ]:
# Rename the values for Electric Vehicle Type
df_full['Electric Vehicle Type'] = df_full.iloc[:, 8].map(lambda x: 'PHEV' if 'PHEV' in x else 'BEV')

# Rename the values for Clean Alternative Fuel Vehicle (CAFV) Eligibility
CAFV_dico = {
    'Eligibility unknown as battery range has not been researched': 'not_verified_yet',
    'Clean Alternative Fuel Vehicle Eligible' : 'eligible',
    'Not eligible due to low battery range' : 'not eligible'
}

df_full['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = df_full['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].map(CAFV_dico)

# <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 3. Dataset Inspection and Cleaning</p>

## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 3.1 Inspection </p>

In [ ]:
df_full.info()

In [ ]:

plt.figure(figsize=(12, 5))
sns.boxplot(df_full, y='Electric Range', x='Model Year')
plt.xticks(rotation=90)
plt.show()

The obseravtions here is contrary to my expectations.
* The electric range of the cars kept increasing over the yeras and topped in 2020. It the started decreasing. Which does not align with technology and innovation.

In [ ]:

for e, EV_Type in enumerate(['PHEV', 'BEV'], start=1):
    plt.figure(figsize=(12, 5))
    plt.subplot(2, 1, e)
    sns.boxenplot(df_full[df_full['Electric Vehicle Type']==EV_Type], y='Electric Range', x='Make')
    plt.xticks(rotation=90)
    plt.title(f'Electric Range by Car Make for {EV_Type}')
    plt.show()

In [ ]:
df_full.describe(exclude='number').T

## What comes out of the data description

### Geographical
* There are 45 US states in the report,
* There are 205 counties,
* There are 770 cities,
* Only 74 electric utilities. Which is less than one for every ten cities on average.

### Makes and Models
* There are 42 makes,
* There are 152 models.


# <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4. EDA of all the EVs </p>


## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.1 Counting </p>

In [ ]:
count_EV_types = df_full['Electric Vehicle Type'].value_counts()

count_EV_types.to_frame(name='Vehicle by EV Type').style.bar(cmap='Greens')

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
count_EV_types.plot.pie(autopct='%.2f%%', radius=1.3,
                        explode=[0.1, 0.05],colors=elec_type_colors,
                        pctdistance=0.5, labeldistance=0.7)
plt.ylabel('')
plt.subplot(1,2,2)
ax = sns.countplot(df_full, x='Electric Vehicle Type', palette=['gold', 'green'])
for count in ax.containers:
    ax.bar_label(count)
plt.yticks([])
plt.suptitle('Count by electric type in the USA', fontsize=18)
plt.show()

Battery Electric Vehicles(BEV) are more popular than Plug_in Hybrid Electric Vehicles(PHEV). In subsequent steps they will be refered to as: BEV and PHEV.

In [ ]:
df_full['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts(normalize=True).to_frame()

cafv_s = df_full['Clean Alternative Fuel Vehicle (CAFV) Eligibility']
cafv_counts = cafv_s.value_counts().to_frame()#.style.bar(cmap='Greens')
cafv_counts['%tage'] = cafv_s.value_counts(normalize=True)*100
cafv_counts.style.background_gradient(cmap='Greens', axis=0)

cafv_counts

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
cafv_counts['count'].plot.pie(autopct='%.2f%%', radius=1.3,
                        colors=['lightgrey', 'grey', 'white'],
                        pctdistance=0.5, labeldistance=0.7)
plt.ylabel('')
plt.subplot(1,2,2)
ax = sns.countplot(df_full, x='Clean Alternative Fuel Vehicle (CAFV) Eligibility', palette='gray')
for count in ax.containers:
    ax.bar_label(count)
plt.yticks([])
plt.xlabel('CAFV')
plt.suptitle('Count of CAFV in the USA', fontsize=18)
plt.show()

34 % of the vehicles are eligible for alternative fuel, while 10 % are not.

## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2 Mapping </p>

In [ ]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
USA = world.loc[world['iso_a3']=='USA']

In [ ]:
# Convert the WKT strings to geometry
gdf_full = df_full.copy()
gdf_full['Vehicle Location'] = gdf_full['Vehicle Location'].apply(lambda x: str(x)).apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf_full = gpd.GeoDataFrame(gdf_full, geometry='Vehicle Location', crs='EPSG:32630')

In [ ]:
red_square = Polygon([(-115.5,  50), (-115.5, 44), (-126, 44), (-126, 50)])
poly_gdf = gpd.GeoDataFrame(geometry=[red_square], crs=world.crs)

In [ ]:
gdf_full['lon'] = gdf_full['Vehicle Location'].apply(lambda p: p.x)
gdf_full['lat'] = gdf_full['Vehicle Location'].apply(lambda p: p.y)

gdf_full[['lon', 'lat']].head()

In [ ]:
# Create the map
map_usa = folium.Map(location=[39.5, -99.7], tiles='cartodbpositron', zoom_start=4.4)

# Add points to the map
mc = MarkerCluster()
for idx, row in gdf_full.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['lon']]))
map_usa.add_child(mc)

# Display the map
map_usa

In [ ]:
# Create the map
map_usa = folium.Map(location=[39.5, -99.7], tiles='cartodbpositron', zoom_start=4.4)

# Set of EVs that left the state of WA
gdf_left = gdf_full[gdf_full['State']!='WA']

def color_scheem(val):
    if val == 'BEV':
        return 'green'
    else:
        return 'yellow'

for i in range(0, len(gdf_left)):
    folium.Circle(
        location= (gdf_left.iloc[i]['lat'], gdf_left.iloc[i]['lon']),
        radius=4000,
        color=color_scheem(gdf_left.iloc[i]['Electric Vehicle Type'])
    ).add_to(map_usa)

# Diplay th map
map_usa

### How many EVs left the state of Washington

In [ ]:
df_full['WA_vs_Other'] = df_full['State'].apply(lambda x: 'stayed in WA' if x=='WA' else 'left WA')

df_full['WA_vs_Other'].value_counts().plot.pie(radius=1.3,explode=[0.6, 0.6], colors=['green', 'maroon'], pctdistance=0.5, labeldistance=0.7)
plt.axis('off')
plt.title('Counts: WA vs left WA', fontsize=18)
plt.show()

In [ ]:
# Set of the cars that left the state
df_left = df_full[df_full['State']!='WA']
print('{} EVs left the state of Washington to other states. This is just about {:.3f} % of the EVs registered there.'.format(df_left.shape[0], 100*df_left.shape[0]/df_full.shape[0]))

### How many for each state

In [ ]:
df_left['State'].value_counts().plot.bar(figsize=(12,5), color='darkgreen',
                                         title='Counts of cars that left the state of washington')
plt.show()

California, despite having less than 120 cars, is the state that received most of the cars that left the state of registration.  

In [ ]:
df_left['State'].value_counts(normalize=True).to_frame().head(10).style.background_gradient(cmap='YlGn')

### 26.24 % of those EVs wen to California, 13.12 % to Virginia

In [ ]:
ax_1 = USA.plot(figsize=(18, 6), color='grey')
gdf_full['Vehicle Location'].plot(color='yellow', ax=ax_1, markersize=8, edgecolor='green')
poly_gdf.boundary.plot(ax=ax_1, color='maroon', linestyle=':')
plt.axis('off')
plt.title('Presence of electric Vehicles across the USA', fontsize=10)
plt.show()

print('The area in the red box account for more than 99% of the electric cars in this dataset')

# <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4. Washington State is our Concern  </p>

### Let's create a Washington only dataset.

In [ ]:
# Filter the dataset to keep only data from Washington state
df_WA = df_full.query("State in ['WA']")
gdf_WA = gdf_full[gdf_full['State']=='WA']

## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.1 Gross Counts of EV that stayed in Washington state</p>

In [ ]:
# Number of Electric Cars in Washington State

print(f'There are {df_WA.shape[0]} electric cars reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_electriv_otilities = df_WA['Electric Utility'].nunique()
print(f'There are {count_electriv_otilities} electric utilities reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_counties = df_WA['County'].nunique()
print(f'There are {count_counties} counties with electric cars reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_cities = df_WA['City'].nunique()
print(f'There are {count_cities} cities with electric cars reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_makes = df_WA['Make'].nunique()
print(f'There are {count_makes} makes of electric cars reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_model = df_WA['Model'].nunique()
print(f'There are {count_model} models of electric cars reported in WA state')

In [ ]:
# Number of Electric Utilities in Washington State
count_postal_codes = df_WA['Postal Code'].nunique()
print(f'There are {count_postal_codes} postal codes reported with electric cars in WA state')

## <p style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2 Detail Counts in Washington state </p>

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.1 Count by Electric Vehicle Type </span>

In [ ]:
count_elect_type = pd.pivot_table(df_WA, values='VIN (1-10)' ,
                       index=['Electric Vehicle Type'],
                       aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_count_elect_type = count_elect_type.style.format("{:d}").background_gradient(cmap='YlGn', axis=0).set_caption('Count of Electric Type')

# Display the styled pivot table
styled_count_elect_type

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
count_elect_type['count'].plot.pie(autopct='%.2f%%', radius=1.3,
                        explode=[0.1, 0.05],colors=elec_type_colors,
                        pctdistance=0.5, labeldistance=0.7)
plt.ylabel('')
plt.subplot(1,2,2)
ax = sns.countplot(df_WA, x='Electric Vehicle Type', palette=['gold', 'green'])
for count in ax.containers:
    ax.bar_label(count)
plt.yticks([])
plt.suptitle('count by electric type in Washington state', fontsize=18)
plt.show()

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.2 Count by Model Year</span>

In [ ]:
table_count_model_year = pd.pivot_table(df_WA, values='VIN (1-10)' ,
                       index=['Model Year'],
                       aggfunc={'count'}, fill_value=0)

table_count_model_year.style.bar(cmap='RdYlGn')

In [ ]:
table_count_model_year.plot.bar(color='green', figsize=(10, 4))
plt.title('Electric vehicle counted by year', fontsize=16)
plt.show()

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.3 Count by Model Year and Electric Type</span>

In [ ]:
table_year_type = pd.pivot_table(df_WA, values='VIN (1-10)' ,
                       index=['Model Year'],
                       columns = ['Electric Vehicle Type'],
                       aggfunc={'count'}, fill_value=0)

In [ ]:
table_year_type.style.bar(cmap='RdYlGn')

In [ ]:
table = pd.pivot_table(df_WA, values='VIN (1-10)' , columns=['Make'],
                       index=['Model Year'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table = table.style.format("{:d}").background_gradient(cmap='YlGnBu', axis=1)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table

Year 2023 has the largest number of electric cars, both BEV and PHEV.

In [ ]:
table_year_type['count'].plot.bar(color =elec_type_colors, figsize=(8, 4))
plt.show()

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.4 Count by Make and Electric Type</span>

In [ ]:
table_make_type = pd.pivot_table(df_WA, values='VIN (1-10)' ,
                       index=['Make'], columns = ['Electric Vehicle Type'],
                       aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table_make_type = table_make_type.style.format("{:d}").bar(axis=0, cmap='Greens')#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table_make_type

In [ ]:
plt.figure(figsize=(16, 14))
for m, Make in enumerate(table_make_type.index, start=1):
    plt.subplot(6, 7, m)
    pd.Series({' ': 1}).plot.pie(colors=['grey'], radius=0.2, shadow=False)
    table_make_type.loc[Make, :]['count'].plot.pie(autopct='%.1f%%', radius=1.2, pctdistance=0.44, shadow=False,
                                         textprops={'color':'black', 'rotation':True, 'weight':'bold', 'size': 8},
                                         startangle=90, colors=elec_type_colors , rotatelabels=True,
                                         labeldistance=0.7, wedgeprops={'width':0.9}, frame=True)
    plt.ylabel('')
    plt.title(Make, color='black', fontsize=10, weight='bold')
plt.tight_layout()

We will focus on the top ten car makes.

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.4 Count Within Top 10 Makes</span>

In [ ]:
Top_10_Makes = list(df_WA['Make'].value_counts().head(10).index)

df_WA_10 = df_WA.query("Make in ['TESLA','CHEVROLET','NISSAN','FORD','KIA','BMW','TOYOTA','HYUNDAI','RIVIAN','VOLKSWAGEN']")

In [ ]:
table = pd.pivot_table(df_WA_10, values='VIN (1-10)' , columns=['Make'],
                       index=['County'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table = table.style.format("{:d}").background_gradient(cmap='YlGn', axis=1)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table

In [ ]:
table = pd.pivot_table(df_WA_10, values='VIN (1-10)' , columns=['Make'],
                       index=['County'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table = table.style.format("{:d}").bar(cmap='YlGn', axis=1)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.5 Count Within counties</span>

In [ ]:
table_county_Make = pd.pivot_table(df_WA_10, values='VIN (1-10)' , columns=['Make'],
                       index=['County'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table_county_Make = table_county_Make.style.format("{:d}").background_gradient(cmap='Blues', axis=0).background_gradient(cmap='Greens', axis=0)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table_county_Make

In [ ]:
df_full['Legislative District'].astype('string').map(lambda x: x.split('.')[0]).value_counts().to_frame().style.bar(cmap='RdYlBu')

In [ ]:
table = pd.pivot_table(df_WA_10, values='VIN (1-10)' , columns=['Make'],
                       index=['Legislative District', 'Electric Vehicle Type'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table = table.style.format("{:d}").background_gradient(cmap='YlGn', axis=1)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 4.2.6 Count within cities </span>

In [ ]:
table = pd.pivot_table(df_WA_10, values='VIN (1-10)' , columns=['Make'],
                       index=['City'], aggfunc={'count'}, fill_value=0)

# Apply styling to the pivot table
styled_table = table.style.format("{:d}").background_gradient(cmap='YlGn', axis=0)#.set_caption('Pivot Table with Conditional Formatting')

# Display the styled pivot table
styled_table

# <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 5. Geospatial Analysis in Washington </span>

## <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 5.2 Map distribution in Washington for Electric Type </span>

In [ ]:
gdf_WA_BEV = gdf_WA[gdf_WA['Electric Vehicle Type']=='BEV']
gdf_WA_PHEV = gdf_WA[gdf_WA['Electric Vehicle Type']=='PHEV']

In [ ]:
# # Create the map
# WA_map = folium.Map(location=[47.46, -120.67], tiles='cartodbpositron', zoom_start=7.8)


# def color_scheem(val):
#     if val == 'BEV':
#         return 'green'
#     else:
#         return 'yellow'

# for i in range(0, len(gdf_WA)):
#     folium.Circle(
#         location= (gdf_WA.iloc[i]['lat'], gdf_WA.iloc[i]['lon']),
#         radius=500,
#         color=color_scheem(gdf_WA.iloc[i]['Electric Vehicle Type'])
#     ).add_to(WA_map)

# # Diplay th map
# WA_map

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 5.2.1 Map distribution BEV Type </span>

In [ ]:
# Create the map
WA_map = folium.Map(location=[47.25, -120.67], tiles='cartodbpositron', zoom_start=6.8)

# Add points to the map
mc = MarkerCluster()
for idx, row in gdf_WA_BEV.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['lon']]))
WA_map.add_child(mc)

# Display the map
WA_map

### <span style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > 5.2.1 Map distribution PHEV Type </span>

In [ ]:
# Create the map
WA_map = folium.Map(location=[47.25, -120.67], tiles='cartodbpositron', zoom_start=6.8,)

# Add points to the map
mc = MarkerCluster()
for idx, row in gdf_WA_PHEV.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['lon']]))
WA_map.add_child(mc)

# Display the map
WA_map

### <center style= "font-family: Calibri; font-weight:bold; letter-spacing: 0px; color: yellow; border-radius:5px; font-size:120%; text-align:left;padding:3.0px; background: #3da859 ; border-bottom: 4px solid yellow; border-top: 4px solid k" > So much is left to do on this dataset. Please join me in getting the story out of it.</center>

<center>
<img src="https://storage.googleapis.com/kaggle-datasets-images/5809040/9537118/38378fdb34deae38f31e1eaebfb258cb/dataset-cover.jpg?t=2024-10-03-07-53-08" width="1000"/>
</center>